In [1]:
import torch
import os
import numpy as np
# TORCH_LOGS="+dynamo,inductor,all" and TORCHDYNAMO_VERBOSE=1
os.environ["TORCH_LOGS"] = "+dynamo"
os.environ["TORCHDYNAMO_VERBOSE"] = "1"

In [2]:
def fn(x):
    flag = x.sum().item() # not support by inductor
    # flag = True
    flag = np.random.uniform(0, 1) > 0.5
    # print() # inductor didn't support
    x  = torch.sigmoid(x)
    if flag:
        x = torch.relu(x)
    return x

def custom_comppiler(gm, example_inputs):
    gm.graph.print_tabular()
    return gm.forward
x  = torch.randn(3)
out = fn(x)
print(out)
torch._dynamo.reset()
compiled_fn = torch.compile(fn, backend=custom_comppiler, fullgraph=False)
compiled_fn(x)

tensor([0.3354, 0.4742, 0.7497])
opcode       name    target    args         kwargs
-----------  ------  --------  -----------  --------
placeholder  l_x_    L_x_      ()           {}
call_method  sum_1   sum       (l_x_,)      {}
output       output  output    ((sum_1,),)  {}
opcode         name             target                                                      args                    kwargs
-------------  ---------------  ----------------------------------------------------------  ----------------------  --------
placeholder    l_x_             L_x_                                                        ()                      {}
call_function  wrapped_uniform  <Wrapped function <original uniform>>                       (0, 1)                  {}
call_function  gt               <built-in function gt>                                      (wrapped_uniform, 0.5)  {}
call_function  sigmoid          <built-in method sigmoid of type object at 0x7f63ae239d00>  (l_x_,)                 {

tensor([0.3354, 0.4742, 0.7497])